In [36]:
from keras.utils import to_categorical
from keras.preprocessing.image import load_img
from keras.models import Sequential
from keras.layers import Dense,Convolution2D,Dropout,Flatten,MaxPooling2D
import os
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
from sklearn.preprocessing import LabelEncoder


In [37]:
TrainDir='images/train'
TestDir='images/test'


In [38]:
def createDataFrame(dir):
    imagePath=[]
    labels=[]
    for label in os.listdir(dir):
        for imageName in os.listdir(os.path.join(dir,label)):
            imagePath.append(os.path.join(dir,label,imageName))
            labels.append(label)
        print(label,"Completed")
    return imagePath,labels
    


In [39]:
train=pd.DataFrame()
train['image'],train['label']=createDataFrame(TrainDir)

happy Completed
sad Completed
fear Completed
surprise Completed
neutral Completed
angry Completed
disgust Completed


In [40]:
test=pd.DataFrame()
test['image'],test['label']=createDataFrame(TestDir)

happy Completed
sad Completed
fear Completed
surprise Completed
neutral Completed
angry Completed
disgust Completed


In [41]:
def extractFeatures(images):
    features=[]
    for image in tqdm(images):
        img=load_img(image,color_mode='grayscale')
        img=np.array(img)
        features.append(img)
    features=np.array(features)
    features=features.reshape(len(features),48,48,1)
    return features


In [42]:
trainFeatures=extractFeatures(train['image'])

  0%|          | 0/28821 [00:00<?, ?it/s]

In [43]:
testFeatures=extractFeatures(test['image'])

  0%|          | 0/7066 [00:00<?, ?it/s]

In [44]:
xTrain=trainFeatures/255.0
xTest=testFeatures/255.0

In [45]:
le=LabelEncoder()
le.fit(train['label'])


LabelEncoder()

In [46]:
yTrain=le.transform(train['label'])
yTest= le.transform(test['label'])

In [47]:
yTrain=to_categorical(yTrain,num_classes=7)
yTest=to_categorical(yTest,num_classes=7)


In [48]:
model = Sequential()

model.add(Convolution2D(128,kernel_size=(3,3), activation='relu', input_shape=(48,48,1)))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0,4))

model.add(Convolution2D(128,kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0,4))

model.add(Convolution2D(128,kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0,4))

model.add(Convolution2D(128,kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0,4))

model.add(Flatten())

model.add(Dense(512,activation='relu'))
model.add((Dropout(0,4)))
model. add (Dense (256, activation='relu'))
model. add (Dropout (0.3))
# output Layer
model. add (Dense(7, activation='softmax'))


/Users/shrey/Library/Python/3.11/lib/python/site-packages/keras/src/layers/convolutional/base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


In [49]:
model. compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics=['accuracy'] )

In [50]:
model.fit(x=xTrain, y=yTrain, batch_size=128, epochs=100, validation_data=(xTest, yTest))


Epoch 1/100
226/226 ━━━━━━━━━━━━━━━━━━━━ 154s 675ms/step - accuracy: 0.2439 - loss: 1.8335 - val_accuracy: 0.2768 - val_loss: 1.7558
Epoch 2/100
226/226 ━━━━━━━━━━━━━━━━━━━━ 137s 608ms/step - accuracy: 0.3514 - loss: 1.6221 - val_accuracy: 0.4438 - val_loss: 1.4170
Epoch 3/100
226/226 ━━━━━━━━━━━━━━━━━━━━ 138s 610ms/step - accuracy: 0.4614 - loss: 1.3862 - val_accuracy: 0.4997 - val_loss: 1.3032
Epoch 4/100
226/226 ━━━━━━━━━━━━━━━━━━━━ 138s 610ms/step - accuracy: 0.5075 - loss: 1.2769 - val_accuracy: 0.5079 - val_loss: 1.2766
Epoch 5/100
226/226 ━━━━━━━━━━━━━━━━━━━━ 140s 618ms/step - accuracy: 0.5347 - loss: 1.2200 - val_accuracy: 0.5330 - val_loss: 1.2188
Epoch 6/100
226/226 ━━━━━━━━━━━━━━━━━━━━ 137s 608ms/step - accuracy: 0.5574 - loss: 1.1643 - val_accuracy: 0.5575 - val_loss: 1.1821
Epoch 7/100
226/226 ━━━━━━━━━━━━━━━━━━━━ 144s 635ms/step - accuracy: 0.5889 - loss: 1.0877 - val_accuracy: 0.5409 - val_loss: 1.2222
Epoch 8/100
226/226 ━━━━━━━━━━━━━━━━━━━━ 137s 607ms/step - accuracy: 

In [51]:
model_json = model. to_json()
with open ("emotiondetector.json", 'w') as json_file:
    json_file.write(model_json)
model. save ("emotiondetector.h5")


In [52]:
from keras.models import model_from_json


In [53]:
with open("emotiondetector.json", "r") as json_file:
    model_json = json_file.read()
model = model_from_json(model_json)
model.load_weights("emotiondetector.h5")

In [54]:
label=['angry','disgust','fear','happy','neutral','sad','suprise']